In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install selenium

In [2]:
# !rm -v geckodriver.log_
# !mv -v geckodriver.log geckodriver.log_

In [3]:
import re
from datetime import datetime as dt

from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.options import FirefoxProfile

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [5]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

url = 'https://www.avito.ru/sevastopol/kvartiry/prodam?'# '&pmax=5000000'

profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

In [6]:
class AvitoDownloader:
    
    def __init__(self,profile_path):
        self._options = Options()
        self._options.profile = FirefoxProfile(profile_path) 
        self._options.headless = True

    def load(self,url):         
        driver = webdriver.Firefox(options=self._options)
        
        driver.get(url)
        root = BeautifulSoup(driver.page_source)
        pages = self._get_pages_count(root)
        # print(pages)
        data = self._parse_page(root)

        for p in tqdm(range(2,pages)):
            driver.get(url+f'&p={p}')
            root = BeautifulSoup(driver.page_source)
            data.extend( self._parse_page( root )  )

        driver.quit()    
        
        return pd.DataFrame(data).dropna()


    @classmethod
    def _parse_page(cls,root):
        return [ cls._parse_item(tag) for tag in root.find_all('div',{'data-marker':'item'}) ]
    
    
    @staticmethod
    def _parse_item(tag):
        try:
            return {    
        'avito_id':tag.attrs['data-item-id'],   # https://www.avito.ru/<id>
        'title':tag.find('a',attrs={'itemprop':'url'}).attrs['title'],    
        'price':tag.find('meta',attrs={'itemprop':'price'}).attrs['content'],
         # 'cur':tag.find('meta',attrs={'itemprop':'priceCurrency'}).attrs['content'],
        'adr':tag.find('div',attrs={'data-marker':'item-address'}).text,
        'description':tag.find('meta',attrs={'itemprop':'description'}).attrs['content'],
            }
        except:
            return dict()

    @staticmethod
    def _get_pages_count(root):
        pp = re.sub( r'.*?p=', '', root.find_all('a',{'class':'pagination-page'})[-1].attrs['href'] ) 
        return 1 if not re.match(r'\d{1,3}', pp) else int(pp)    

In [7]:
df = AvitoDownloader(profile_path).load(url)

ts = dt.now().strftime('%Y-%m-%d_%H-%M')
df[['avito_id','title','price','adr','description',]].to_excel(f'data/avito_{ts}_raw.xlsx',index=False)

/tmp/ipykernel_704070/901723822.py:5: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  self._options.profile = FirefoxProfile(profile_path)
/tmp/ipykernel_704070/901723822.py:5: DeprecationWarning: Setting a profile has been deprecated. Please use the set_preference and install_addons methods
  self._options.profile = FirefoxProfile(profile_path)


  0%|          | 0/78 [00:00<?, ?it/s]

In [10]:
class AvitoDataCleanerRealty:
    
    @staticmethod
    def transform(data):
        df = data.copy()
        df['title'] = df['title'].str.lower().str.extract( r'.*«(.*)».*',expand=False)
        df['nrooms'] = df['title'].str.extract( r'.*(\d)-к. .*',expand=False)
        df['floor'] = df['title'].str.extract( r'.*(\d+)/\d+.эт.*',expand=False)
        df['nfloors'] = df['title'].str.extract( r'.*\d+/(\d+).эт.*',expand=False)
        df['area'] = (
            df['title']
            .str.extract( r'.*, ([\d,]+).*м²,.*',expand=False)
            .str.replace(',','.')
        )
        df['is_studio'] = df['title'].str.lower().str.match(r'.*студи.*')
        df['is_apartment'] = df['title'].str.lower().str.match(r'.*апартамент.*')
        df['is_part'] = df['title'].str.lower().str.match(r'.*дол.*')
        df['is_auction'] = df['title'].str.lower().str.match(r'.*аукци.*')
        df['is_openspace'] = df['title'].str.lower().str.match(r'.*своб.*планир.*')
        df['is_roof'] = df['description'].str.lower().str.match('.*мансард.*')
        df['is_SNT'] = (
            df['adr'].str.match('.*СТ .*') 
            | df['adr'].str.match('.*СНТ .*') 
            | df['adr'].str.lower().str.match('.*садов.*')
        )
        df['is_fiolent'] = (
            df['description'].str.lower().str.match('.*фиолент.*')
            | df['adr'].str.lower().str.match('.*фиолент.*')
        )

        # df[ df['adr'].str.match('.*СТ .*') ]
        # df[ df['adr'].str.match('.*СНТ .*') ]
        # df[ df['adr'].str.match('.*садов.*') ]
        # df[ df['description'].str.match('.*садов.*') ]

        # df['avito_id'] = df['avito_id'].astype(int)
        df['price'] = df['price'].astype(int)
        df['nrooms'] = df['nrooms'].fillna('0').astype(int)
        df['floor'] = df['floor'].fillna('0').astype(int)
        df['nfloors'] = df['nfloors'].fillna('0').astype(int)
        df['area'] = df['area'].fillna('0.').astype(float)
        df['priceM'] = df['price']/1e6
        df['is_last_floor'] = ( df['floor'] == df['nfloors'] )

        cols = [
         'adr',
         'title',
         'priceM',
         'nrooms',
         'floor',
         'nfloors',
         'area',
         'is_studio',
         'is_apartment',
         'is_part',
         'is_auction',
         'is_openspace',
         'is_SNT',
         'is_fiolent',   
         'is_last_floor', 
         'is_roof',   
         'description',
         'price',
         'avito_id',
        ]

        return df[cols]


In [13]:
df_ = AvitoDataCleanerRealty.transform(df)
df_

,adr,title,priceM,nrooms,floor,nfloors,area,is_studio,is_apartment,is_part,is_auction,is_openspace,is_SNT,is_fiolent,is_last_floor,is_roof,description,price,avito_id
0,"ЖК «Новый»ш. Лабораторное, д. 33, секц. 4","1-к. квартира, 41,7 м², 9/12 эт.",6.36,1,9,12,41.70,False,False,False,False,False,False,False,False,False,"В продаже просторная однокомнатная квартира-бабочка на 9 этаже двенадцатиэтажного дома в новом комплексе жилых — ЖК «Новый» (вторая очередь), Нахимовский район города Севастополь. \n\nПодробнее о квартире: \n\n— Общая площадь квартиры: 41,72 м²;\n— Кухня с выхо",6358128,2333528475
1,"ЖК «Апельсин» ул. Павла Корчагина, д. 23, блок-секция 1","1-к. квартира, 36 м², 4/9 эт.",7.33,1,4,9,36.00,False,False,False,False,False,False,False,False,False,"Квартира в Готовом Доме! \n\nРассрочка на 3 года. \n\nПродажа по Договору купли-продажи (от Застройщика). \n\nИпотека. \n\nЖК «Апельсин» расположен на улице Корчагина в 10 минутах от моря и самого нового, современного, качественно оборудованного пляжа Севастополя",7334000,2409291266
2,"ЖК «Центр»ул. Генерала Крейзера, д. 8, корп. 5","2-к. квартира, 69 м², 3/10 эт.",7.57,2,3,10,69.00,False,False,False,False,False,False,False,False,False,Статусный современный жилой комплекс бизнес-класса Центр находится в самом сердце города Федерального значения Севастополь. ЖК Центр — жизнь в эпицентре событий. Расположение в историческом центре Ленинского района дарит возможность быстрого доступа до гла,7568000,2468437907
3,"ЖК «Университетский-2»ул. Вакуленчука, д. 28","1-к. квартира, 44 м², 3/7 эт.",7.00,1,3,7,44.00,False,False,False,False,False,False,False,False,False,"В продаже просторная и светлая однокомнатная квартира в лучшем районе города с развитой инфраструктурой. \n\nПодробнее о квартире: \n\n— общая площадь квартиры: 44 м²;\n— кухня: 15,3 м²;\n— комната: 19,3 м²;\n— санузел совмещенный;\n— просторная прихожая. \n\nВысота",7000000,2429030034
4,"ЖК «Доброгород»ул. Токарева, 7 этап, блок-секция 1","3-к. квартира, 76,6 м², 7/9 эт.",10.34,3,7,9,76.60,False,False,False,False,False,False,False,False,False,"Крупнейший в Севастополе семейный микрорайон «Доброгород»! \n\nПобедитель всероссийской премии «Топ ЖК 2021» в номинации «Лучший жилой комплекс». \n\nИпотека с господдержкой, рассрочка от Застройщика. \n\n«Доброгород» по праву считается лучшим жилым кварталом Се",10341000,2505165298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3898,"Севастополь, проспект Победы, 44В","2-к. квартира, 53,3 м², 5/9 эт.",12.00,2,5,9,53.30,False,False,False,False,False,False,False,False,False,Продаю шикарную 2-к квартиру в Нахимовском районе не имеющую аналогов на рынке! В данной квартире выполнен дизайнерский ремонт по проекту в светлых тонах — максимально приятных для восприятия глаз. В квартире выполнена удачная планировка с раздельными комн,12000000,2409273023
3899,"пос. городского типа Кача, ул. Авиаторов, 34","3-к. квартира, 70 м², 1/5 эт.",8.00,3,1,5,70.00,False,False,False,False,False,False,False,False,False,"Продается отличная квартира у моря в поселке городского типа Кача Нахимовский район города-героя Севастополя. \nНаходится в живописном уголке с чистым морским воздухом. \nКвартира очень светлая и теплая в окнах стеклопакеты. \nОбщая площадь квартиры 70 кв. М,",8000000,2366781595
3900,"ул. Дмитрия Ульянова, 11А","2-к. квартира, 145,2 м², 6/7 эт.",36.00,2,6,7,145.20,False,False,False,False,False,False,False,False,False,"Продаётся квартира с панорамным видом и качественным ремонтом для комфортного проживания. Общей площадью 145.2 м. Кв. Расположена на 6-м этаже 7-ми этажного дома ( лифт) 2012г. Постройки. \nКухня-столовая объединена с гостиной, гостевой санузел, отдельная с",36000000,2447926345
3901,"ЖК «Центр»ул. Генерала Крейзера, д. 8, корп. 7","3-к. квартира, 98 м², 4/5 эт.",13.50,3,4,5,98.00,False,False,False,False,False,False,False,False,False,"В г. Севастополь продаётся Видовая квартира в доме Бизнес Класса на ул. Гоголя, 19 в пешей доступности от центра

In [14]:
ts = dt.now().strftime('%Y-%m-%d_%H-%M')
df_.to_excel(f'data/avito_{ts}.xlsx',index=False)

ts

'2022-08-29_13-17'

-----

In [ ]:
# df[ df['description'].str.match(r'.*часть дом.*') ][['title','description','is_part']]
# df[ df['description'].str.match(r'.*\d/\d квартир.*') ][['title','description','is_part']]

# df[ df['area'].isnull() ]
# df[ df['floor'].isnull() ]
# df[ df['nfloors'].isnull() ]
# df.query('is_openspace')

In [ ]:
# def get_pages_count(root):
#     pp = re.sub( r'.*?p=', '', root.find_all('a',{'class':'pagination-page'})[-1].attrs['href'] ) 
#     return 1 if not re.match(r'\d{1,3}', pp) else int(pp)


# def parse_item(tag):
#     try:
#         return {    
#     'avito_id':tag.attrs['data-item-id'],   # https://www.avito.ru/<id>
#     'title':tag.find('a',attrs={'itemprop':'url'}).attrs['title'],    
#     'price':tag.find('meta',attrs={'itemprop':'price'}).attrs['content'],
#      # 'cur':tag.find('meta',attrs={'itemprop':'priceCurrency'}).attrs['content'],
#     'adr':tag.find('div',attrs={'data-marker':'item-address'}).text,
#     'description':tag.find('meta',attrs={'itemprop':'description'}).attrs['content'],
#         }
#     except:
#         return dict()


# options=Options()
# options.profile = FirefoxProfile(profile_path) 
# options.headless = True
# driver = webdriver.Firefox( options=options)

# driver.get(url)
# html = driver.page_source

# root = BeautifulSoup(html)
# pages = get_pages_count(root)
# print(pages)
# data = [ parse_item(tag) for tag in root.find_all('div',{'data-marker':'item'}) ]

# for p in tqdm(range(2,pages)):
#     driver.get(url+f'&p={p}')
#     html = driver.page_source
#     root = BeautifulSoup(html)
#     data.extend( [ parse_item(tag) for tag in root.find_all('div',{'data-marker':'item'}) ] )

# # driver.close()
# driver.quit()

In [ ]:
# import pandas as pd
# df = pd.DataFrame(data).dropna()
# print(len(df))
# df.sample(10)